In [10]:
%pylab --no-import-all
%matplotlib inline
import seaborn as sns
import sys
import pandas as pd
from ipywidgets import widgets
from IPython.display import Markdown, display, clear_output, Audio
import os
from os.path import join, isdir
from glob import iglob
import json
from pprint import pformat
import utils
import pickle
import io
import base64
import birdsonganalysis as bsa

sys.path.append('../model')

from song_model import SongModel

sns.set_palette('colorblind')

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [6]:
gridpath = '../model/res/prior_dev_grid_170403_170705/'
run_paths = [run_path for run_path in iglob(join(gridpath, '*')) if isdir(run_path)]

data_cache = {}
def get_data(i):
    try:
        return data_cache[i]
    except KeyError:
        try:
            with open(join(run_paths[i], 'data.pkl'), 'rb') as f:
                data_cache[i] = pickle.load(f)
        except FileNotFoundError:
            try:
                with open(join(run_paths[i], 'data_cur.pkl'), 'rb') as f:
                    data_cache[i] = pickle.load(f)
            except FileNotFoundError:
                data_cache[i] = None
        return data_cache[i]

In [7]:
with open(join(gridpath, 'desc.md')) as f:
    display(Markdown(f.read()))

# Run description

## Goals

- Big run with all the different parameters to find the best set

## What have changed compared to previous runs?

- I have introduced a gridsearch algorithm

## Expected Results (Success condition)

- One of the parameter set should sound right with both simple and ba_example.

<!-- ## Notes -->


In [35]:
def show_run(i):
    show = [
        show_conf(i),
        show_learning_curve(i),
        widgets.HTML('<h4>Best</h4>'),
        plot_spec(i),
        get_audio(i),
        plot_ab(i)
    ]
    clear_output()
    for item in show:
        display(item)
    
slider = widgets.IntSlider(min=0, max=len(run_paths)-1)
slider.observe(lambda x: show_run(x['new']), names='value')
display(slider)


In [36]:
def show_conf(i):
    with open(join(run_paths[i], 'conf.json')) as f:
        conf = json.load(f)
    label = widgets.HTML('')
    label.value = '<h3>' + conf['name'] + '</h3><pre>'+pformat(conf, compact=True) + '</pre>'
    return label

def plot_to_html(fig):
    # write image data to a string buffer and get the PNG image bytes
    buf = io.BytesIO()
    fig.tight_layout()
    fig.savefig(buf, format='png')
    buf.seek(0)
    return widgets.HTML("""<img src='data:image/png;base64,{}'/>""".format(
        base64.b64encode(buf.getvalue()).decode('ascii')))

def show_learning_curve(i):
    data = get_data(i)
    if data is None:
        return
    fig = plt.figure(figsize=(13, 4))
    ax = fig.gca()
    ax.set_ylim((0, 900))
    ax = utils.draw_learning_curve(data, ax)
    plt.close(fig)
    return plot_to_html(fig)

def plot_spec(i):
    data = get_data(i)
    root_data = [item[1] for item in data if item[0] == 'root']
    rd = pd.DataFrame(root_data)
    best = np.argmax(rd['scores'].iloc[-1])
    s = rd['songs'].iloc[-1][best].gen_sound()
    fig = plt.figure(figsize=(13, 4))
    ax = fig.gca()
    bsa.spectral_derivs_plot(bsa.spectral_derivs(s, 256, 40, 1024), contrast=0.01, ax=ax)
    plt.close(fig)
    return plot_to_html(fig)

def get_audio(i):
    data = get_data(i)
    root_data = [item[1] for item in data if item[0] == 'root']
    rd = pd.DataFrame(root_data)
    best = np.argmax(rd['scores'].iloc[-1])
    s = rd['songs'].iloc[-1][best].gen_sound()
    a = Audio(s, rate=44100)
    return a

def plot_ab(i):
    data = get_data(i)
    root_data = [item[1] for item in data if item[0] == 'root']
    rd = pd.DataFrame(root_data)
    best = np.argmax(rd['scores'].iloc[-1])
    ab = rd['songs'].iloc[-1][best].gen_alphabeta()
    fig = plt.figure(figsize=(13, 4))
    ax = fig.gca()
    ax.plot(ab)
    plt.close(fig)
    return plot_to_html(fig)